1. PIPELINE可以拦截item，之后可以对item进行诸如保存到数据库或者任何我们想要保存的地方之类的操作
    * 针对item对象的操作方法体被定义在PIPELINE类中的process_item()方法里
2. 创建一个PIPLINE，功能为把item以JSON格式进行保存
```
import codecs
import json
class JsonWithEncodingPipeline(object):
    # 自定义json文件的导出
    def __init__(self):
        self.file = codecs.open('article.json', 'w', encoding="utf-8")

    def process_item(self, item, spider):
        lines = json.dumps(dict(item), ensure_ascii=False) + "\n"
        self.file.write(lines)
        return item

    def spider_closed(self, spider):
        self.file.close()
```
    * codecs.open()和普通的open()相比,codecs.open()能帮我们处理编码方面的繁杂工作
    * lines = json.dumps(dict(item), ensure_ascii=False)
        * json.dumps():把Python中的dict类型转换成JSON类型
        * json.loads():把JSON类型转换成Python中的dict类型
        * ensure_ascii=False:确保在导入中文以及其他字符编码时不会出错
3. 在PIPLINE处理完item之后不要忘记返回item

1. Scrapy本身提供了把item导出成各种文件格式的机制
    * scrapy.exporters
    ```
    __all__ = ['BaseItemExporter', 'PprintItemExporter', 'PickleItemExporter',
           'CsvItemExporter', 'XmlItemExporter', 'JsonLinesItemExporter',
           'JsonItemExporter', 'MarshalItemExporter']
    ```
    * 以from scrapy.exporters import JsonItemExporter为例
    ```
    from scrapy.exporters import JsonItemExporter
    
    class JsonExporterPipleline(object):
        # 调用scrapy提供的json export导出json文件
        def __init__(self):
            self.file = open('articleexport.json', 'wb')
            self.exporter = JsonItemExporter(self.file, encoding="utf-8", ensure_ascii=False)
            self.exporter.start_exporting()

        def close_spider(self, spider):
            self.exporter.finish_exporting()
            self.file.close()

        def process_item(self, item, spider):
            self.exporter.export_item(item)
            return item
    ```